In [ ]:
%pip install matplotlib
%pip install seaborn
#%pip install -U memory_profiler
%pip install -U scikit-learn scipy matplotlib
from openfhe import *
import matplotlib.pyplot as plt
import time
import timeit

In [ ]:
import math
import cmath
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

def calculate_approximation_error(result,expected_result):
    if len(result) != len(expected_result):
        raise Exception("Cannot compare vectors with different numbers of elements")
    # using the infinity norm
    # error is abs of the difference of real parts
    max_error = max([abs(el1.real - el2.real) for (el1, el2) in zip(result, expected_result)])
    # return absolute value of log base2 of the error
    return abs(math.log(max_error,2))

def ErrorCalculate():
    ErrorListPrecision=list()
    ylist=list()
    ErrorListPrecision=list()
    ErrorListRelativ=list()
    Precisionlist=list()
    Iterationlist=list()
    PrintErrorList=list()
    mult_depth = 5
    scale_mod_size =59
    batch_size = 8
    #RingDimValue=2**RingDim
    parameters = CCParamsCKKSRNS()
    parameters.SetMultiplicativeDepth(mult_depth)
    parameters.SetScalingModSize(scale_mod_size)
    parameters.SetBatchSize(batch_size)
    #parameters.SetSecurityLevel(HEStd_NotSet); # disable security HEStd_NotSet GetSecurityLevel HEStd_256_classic HEStd_128_classic
    #parameters.SetRingDim( RingDimValue );
            
    cc = GenCryptoContext(parameters)
    cc.Enable(PKESchemeFeature.PKE)
    cc.Enable(PKESchemeFeature.KEYSWITCH)
    cc.Enable(PKESchemeFeature.LEVELEDSHE)
    print("The CKKS scheme is using ring dimension: " + str(cc.GetRingDimension()))
            
    keys = cc.KeyGen()
    cc.EvalMultKeyGen(keys.secretKey)
    cc.EvalRotateKeyGen(keys.secretKey, [1, -2])
            
    x1 =[5.215, 5.515, 5.715, 5.015, 5.016, 5.015, 4.015, 5.015] 
    x2 =[5.215, 5.515, 5.715, 5.015, 5.016, 5.015, 4.015, 5.015]
        
    ptx1 = cc.MakeCKKSPackedPlaintext(x1)
    ptx2 = cc.MakeCKKSPackedPlaintext(x2)
        
    c1 = cc.Encrypt(keys.publicKey, ptx1)
    c2 = cc.Encrypt(keys.publicKey, ptx2)
    for x in range(0,17):
        print("Tiefe", x)
        c_Mult = cc.EvalAdd(c1, c2)
        precision = 15
        result = cc.Decrypt(c_Mult, keys.secretKey)
        result.SetLength(batch_size)
        print(str(result))
        x1= [x * y for x, y in zip(x1, x2)]
        print(" Ergebnis")
        #print(x1)
        print(result.GetCKKSPackedValue())
        pt1 = cc.MakeCKKSPackedPlaintext(x1)
        precision = calculate_approximation_error(result.GetCKKSPackedValue(),pt1.GetCKKSPackedValue())
  
        print(f"precision after 1 iteration: {precision} bits\n")
        Precisionlist.append(result.GetLogPrecision())
        Iterationlist.append(x)
        ErrorList=list()
        Yerr=list()
        for y in range(0,batch_size): 
            Entcrypted=result.GetCKKSPackedValue()[y]
            ylist.append(Entcrypted.real)
            ErrorList.append(abs(Entcrypted.real-x1[y]))
    #ErrorListMean.append(abs(np.mean(ErrorList)))
        print(ErrorList)
        PrintErrorList.append((np.mean(ErrorList)))
        print(PrintErrorList)
        for y in range(0,batch_size): 
            Entcrypted=result.GetCKKSPackedValue()[y]
            ylist.append(Entcrypted.real)
            ErrorListRelativ.append(((abs(Entcrypted.real-x1[y]))/x1[y])*100)
        print("Relativ")
        #############PrintErrorList.append((np.mean(ErrorListRelativ)))
        print(ErrorListRelativ)
        #Scalelist.append(Scale)
        #ErrorListPrecisionMean.append(np.mean(ErrorListPrecision))
        ptx = cc.MakeCKKSPackedPlaintext(result.GetCKKSPackedValue())
        c1=cc.Encrypt(keys.publicKey, ptx)
        ErrorListRelativ.clear()
      
                   
    print("#--------------------------------------------------------------------------------------")
    plt.xlabel("Iterationen")
    plt.ylabel("Präzisionsbit ")
    plt.plot(Iterationlist, Precisionlist, marker="." )
    #plt.plot(Iterationlist, PrintErrorList, marker="." )
    plt.show()




    
        

In [ ]:
import math
import cmath
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

def calculate_approximation_error(result,expected_result):
    if len(result) != len(expected_result):
        raise Exception("Cannot compare vectors with different numbers of elements")
    # using the infinity norm
    # error is abs of the difference of real parts
    max_error = max([abs(el1.real - el2.real) for (el1, el2) in zip(result, expected_result)])
    # return absolute value of log base2 of the error
    return abs(math.log(max_error,2))


def CKKSKryptADDErrorScale211():
    ErrorList=list()
    ErrorListPrecision=list()
    ErrorListRelativ=list()
    ErrorListMean=list()
    ErrorListPrecisionMean=list()
    ErrorListRelativMean=list()
    ylist=list()
    Scalelist=list()
    ScaleNewlist=list()
    ErrorNewListMean=list()
    RingdimvalueList = list()
    TimeList = list()
    for Scale in range(45, 51, 1):
        print(Scale)
        mult_depth = 1
        scale_mod_size =Scale
        batch_size = 8
        RingDimValue=2**15
        parameters = CCParamsCKKSRNS()
        parameters.SetMultiplicativeDepth(mult_depth)
        parameters.SetScalingModSize(scale_mod_size)
        parameters.SetBatchSize(batch_size)
        parameters.SetSecurityLevel(HEStd_NotSet); # disable security HEStd_NotSet GetSecurityLevel HEStd_256_classic HEStd_128_classic
        parameters.SetRingDim( RingDimValue );
        
        cc = GenCryptoContext(parameters)
        cc.Enable(PKESchemeFeature.PKE)
        cc.Enable(PKESchemeFeature.KEYSWITCH)
        cc.Enable(PKESchemeFeature.LEVELEDSHE)
        print("The CKKS scheme is using ring dimension: " + str(cc.GetRingDimension()))
        
        keys = cc.KeyGen()
        cc.EvalMultKeyGen(keys.secretKey)
        cc.EvalRotateKeyGen(keys.secretKey, [1, -2])
        
        x1 =[5.215, 5.515, 5.715, 5.015, 5.016, 5.015, 4.015, 5.015] #[2**20+0.251, 2**21+0.0251, 2**22 + 0.251, 2**23 + 0.0251, 2**24+0.251, 2**25+0.0251, 2**26+0.251, 2**27+ 0.0251]#[0.25111, 20.025111, 2.25111, 2.025111, 2.25111, 2.025111, 2.25111, 2.025111]#[20+0.251, 21+0.0251, 22 + 0.251, 23 + 0.0251, 24+0.251, 25+0.0251, 26+0.251, 27+ 0.0251]#
        x2 =[5.215, 5.515, 5.715, 5.015, 5.016, 5.015, 4.015, 5.015]#[5.0, 4.0, 3.0, 2.0, 1.0, 0.7, 0.5, 0.2]#[2**20+0.251, 2**21+0.0251, 2**22 + 0.251, 2**23 + 0.0251, 2**24+0.251, 2**25+0.0251, 2**26+0.251, 2**27+ 0.0251]#[0.25111, 20.025111, 2.25111, 2.025111, 2.25111, 2.025111, 2.25111, 2.025111]#[20+0.251, 21+0.0251, 22 + 0.251, 23 + 0.0251, 24+0.251, 25+0.0251, 26+0.251, 27+ 0.0251]#[2**20+0.251, 2**21+0.0251, 2**22 + 0.251, 2**23 + 0.0251, 2**24+0.251, 2**25+0.0251, 2**26+0.251, 2**27+ 0.0251]#[2**20, 2**21, 2**22, 2**23, 2**24, 2**25, 2**26, 2**27]
    
        ptx1 = cc.MakeCKKSPackedPlaintext(x1)
        ptx2 = cc.MakeCKKSPackedPlaintext(x2)
    
        c1 = cc.Encrypt(keys.publicKey, ptx1)
        c2 = cc.Encrypt(keys.publicKey, ptx2)
        c_add = cc.EvalMult(c1, c2)
        precision = 15
        result = cc.Decrypt(c_add, keys.secretKey)
        result.SetLength(batch_size)
        unencryptedResult= [x * y for x, y in zip(x1, x2)] 
        ErrorList=list()
        Yerr=list()
        for y in range(0,batch_size): 
            Entcrypted=result.GetCKKSPackedValue()[y]
            ylist.append(Entcrypted.real)
            ErrorList.append(abs(Entcrypted.real-unencryptedResult[y]))
        ErrorListMean.append(np.mean(np.abs(ErrorList)))
        #print(ErrorListMean)
        ErrorList.clear()
        #print("Estimated precision in bits: " + str(result.GetLogPrecision()))
        for y in range(0,batch_size): 
            Entcrypted=result.GetCKKSPackedValue()[y]
            ylist.append(Entcrypted.real)
            ErrorListRelativ.append((100-(Entcrypted.real/unencryptedResult[y])*100))

    
        for y in range(0,batch_size): 
            x=result.GetCKKSPackedValue()[y]
            #Yerr.append(x.real+ErrorList[y])

        for y in range(0,batch_size): 
            x=result.GetCKKSPackedValue()[y]
            #ErrorListPrecision.append(math.log((abs(x.real-unencryptedResult[y])),2))
            #ErrorListPrecision.append(math.log2(abs(x.real-unencryptedResult[y])))
        Scalelist.append(Scale)
        ErrorListPrecisionMean.append(np.mean(ErrorListPrecision))
        #print(ErrorListPrecision)
        #ErrorListPrecision=[]
   
    sns.set(style="whitegrid")
    plt.figure(figsize=(11, 7))
    print("Estimated precision in bits: " + str(result.GetLogPrecision()))
    #plt.plot(Scalelist, ErrorListMean, label ='y1')

    plt.bar(Scalelist, ErrorListMean, 
        width = 0.4)

    plt.xlabel("Iterationen ")
    plt.ylabel("Absoluter Fehler ")
    plt.show()
    
   



    
        

In [ ]:
import math
import cmath
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

def calculate_approximation_error(result,expected_result):
    if len(result) != len(expected_result):
        raise Exception("Cannot compare vectors with different numbers of elements")
    # using the infinity norm
    # error is abs of the difference of real parts
    max_error = max([abs(el1.real - el2.real) for (el1, el2) in zip(result, expected_result)])
    # return absolute value of log base2 of the error
    return abs(math.log(max_error,2))


def CKKSKryptADDErrorScale211():
    ErrorList=list()
    ErrorListPrecision=list()
    ErrorListRelativ=list()
    ErrorListMean=list()
    ErrorListPrecisionMean=list()
    ErrorListRelativMean=list()
    ylist=list()
    Scalelist=list()
    ScaleNewlist=list()
    ErrorNewListMean=list()
    RingdimvalueList = list()
    TimeList = list()
    for Scale in range(45, 51, 1):
        print(Scale)
        mult_depth = 1
        scale_mod_size =Scale
        batch_size = 8
        RingDimValue=2**15
        parameters = CCParamsCKKSRNS()
        parameters.SetMultiplicativeDepth(mult_depth)
        parameters.SetScalingModSize(scale_mod_size)
        parameters.SetBatchSize(batch_size)
        parameters.SetSecurityLevel(HEStd_NotSet); # disable security HEStd_NotSet GetSecurityLevel HEStd_256_classic HEStd_128_classic
        parameters.SetRingDim( RingDimValue );
        
        cc = GenCryptoContext(parameters)
        cc.Enable(PKESchemeFeature.PKE)
        cc.Enable(PKESchemeFeature.KEYSWITCH)
        cc.Enable(PKESchemeFeature.LEVELEDSHE)
        print("The CKKS scheme is using ring dimension: " + str(cc.GetRingDimension()))
        
        keys = cc.KeyGen()
        cc.EvalMultKeyGen(keys.secretKey)
        cc.EvalRotateKeyGen(keys.secretKey, [1, -2])
        
        x1 =[5.215, 5.515, 5.715, 5.015, 5.016, 5.015, 4.015, 5.015] 
        x2 =[5.215, 5.515, 5.715, 5.015, 5.016, 5.015, 4.015, 5.015]]
    
        ptx1 = cc.MakeCKKSPackedPlaintext(x1)
        ptx2 = cc.MakeCKKSPackedPlaintext(x2)
    
        c1 = cc.Encrypt(keys.publicKey, ptx1)
        c2 = cc.Encrypt(keys.publicKey, ptx2)
        c_add = cc.EvalMult(c1, c2)
        precision = 15
        result = cc.Decrypt(c_add, keys.secretKey)
        result.SetLength(batch_size)
        unencryptedResult= [x * y for x, y in zip(x1, x2)]
        ErrorList=list()
        Yerr=list()
        for y in range(0,batch_size): 
            Entcrypted=result.GetCKKSPackedValue()[y]
            ylist.append(Entcrypted.real)
            ErrorList.append(abs(Entcrypted.real-unencryptedResult[y]))
        ErrorListMean.append(np.mean(np.abs(ErrorList)))
        ErrorList.clear()
        #print("Estimated precision in bits: " + str(result.GetLogPrecision()))
        for y in range(0,batch_size): 
            Entcrypted=result.GetCKKSPackedValue()[y]
            ylist.append(Entcrypted.real)
            ErrorListRelativ.append((100-(Entcrypted.real/unencryptedResult[y])*100))

    
        for y in range(0,batch_size): 
            x=result.GetCKKSPackedValue()[y]
            #Yerr.append(x.real+ErrorList[y])

        for y in range(0,batch_size): 
            x=result.GetCKKSPackedValue()[y]
            #ErrorListPrecision.append(math.log((abs(x.real-unencryptedResult[y])),2))
            #ErrorListPrecision.append(math.log2(abs(x.real-unencryptedResult[y])))
        Scalelist.append(Scale)
        ErrorListPrecisionMean.append(np.mean(ErrorListPrecision))
     
    sns.set(style="whitegrid")
    plt.figure(figsize=(11, 7))
    print("Estimated precision in bits: " + str(result.GetLogPrecision()))
    #plt.plot(Scalelist, ErrorListMean, label ='y1')

    plt.bar(Scalelist, ErrorListMean, 
        width = 0.4)

    plt.xlabel("Iterationen ")
    plt.ylabel("Absoluter Fehler ")
    plt.show()
    
   



    
        